In [4]:
import os
import sys
import re
import collections
import sqlparse
import prettytable
import copy

In [9]:
# main_script_path = os.path.dirname(os.path.realpath(__file__))
# data_path=main_script_path
data_path=""
schema={} ##dictionary of tuples where each tuple belongs to a table which conatins all its column names
dataset=collections.defaultdict(list) ##dictionary in which table is key and its value is a list which contains each of its row as dict
AGGREGATE_FUNCTIONS=("avg", "max", "min", "sum")
CONDITIONAL_TUPLE=("AND", "OR", "(", ")")
OPERATOR_TUPLE=("=", ">", "<", "!=", "<=", ">=")

In [10]:
def quotes_removal(s): ## to remove "" or '' from values in table.
    ### return quotes(single or double) free value (integer or variable/column name)
    s = s.strip()
    while len(s) > 1 and (s[0]=='\"' or s[0]=='\'') and s[0]==s[-1]:
        s = s[1:-1] ## copying original str say,"abcxyz" or 'abcxyz' to abcxyz
    return s

In [11]:
def generate_schema_and_load_data(filename):
##Generate the table structure using database schema
##here filename is "metadata.txt"
    with open(os.path.join(data_path,filename), "r") as f:
        table_found=-1
        for row in f.readlines():
            row=quotes_removal(row) ## to remove "" or '' from each row of metadata.txt
            #print len(row) 
            if row=="<begin_table>":
                table_found=0
            elif row=="<end_table>":
                table_found=-1
            else:
                if table_found==0:
                    schema[row]=()
                    #print schema[row]
                    table_found=row #table name
                    #print table_found
                else: ##Assuming a column name is a capital english alphabet
                    schema[table_found]+=(table_found+"."+row,) ##only one element in tuple
                    #print schema[table_found]
    f.close()
    #print "Database Schema Created Using Metadata"

    files=[] #contains table filenames
    for filename in os.listdir(data_path):
        temp=[]
        temp=filename.split(".")
        if len(temp)==2:
            if temp[1]=="csv":
                if temp[0] in schema.keys():
                    table_filename=temp[0]
                    for z in range(1,len(temp)):
                        table_filename+="."+temp[z]
                    files.append(table_filename)

    ## Loading data from all the files
    #print "Data is loading from each table files"
    for filename in files:
        #print "Data loaded from "+str(filename)
        with open(os.path.join(data_path,filename),"r") as f:
            for row in f.readlines():
                split_row=row.split(",")
                tablename=filename.split(".")[0]
                #print schema[tablename]
                #print split_row
                #print zip(schema[tablename],split_row)
                each_row_info=zip(schema[tablename],split_row)
                values=[]
                columns=[]
                for i in each_row_info:
                    j=list(i)
                    if len(quotes_removal(j[1]))>0: ###if value is present
                        values.append(int(quotes_removal(j[1])))
                        columns.append(j[0])
                    else: ## if value is missing
                        values.append(0) ##putting zero in the empty space
                        columns.append(j[0])
                temp_dict={}
                #print values
                #print columns
                for x in range(0,len(columns)):
                    temp_dict[columns[x]]=values[x]
                #print temp_dict
                dataset[tablename].append(temp_dict)
        f.close()

In [12]:
metadata_filename="metadata.txt"
generate_schema_and_load_data(metadata_filename)
input_query=sys.argv[1]
print (input_query)
formatted_query=sqlparse.format(input_query,reindent=True,keyword_case='upper')
print (formatted_query)

FileNotFoundError: [Errno 2] No such file or directory: ''